In [1]:
import pandas as pd

In [2]:
file_paths = [f'/Users/TuanKiet/Desktop/DSLearn/vnstock_project/data/outputs/output_daily_{i}-12-2023.xlsx' for i in range(25, 30)]
file_paths

['/Users/TuanKiet/Desktop/DSLearn/vnstock_project/data/outputs/output_daily_25-12-2023.xlsx',
 '/Users/TuanKiet/Desktop/DSLearn/vnstock_project/data/outputs/output_daily_26-12-2023.xlsx',
 '/Users/TuanKiet/Desktop/DSLearn/vnstock_project/data/outputs/output_daily_27-12-2023.xlsx',
 '/Users/TuanKiet/Desktop/DSLearn/vnstock_project/data/outputs/output_daily_28-12-2023.xlsx',
 '/Users/TuanKiet/Desktop/DSLearn/vnstock_project/data/outputs/output_daily_29-12-2023.xlsx']

In [3]:
all_sheets = pd.read_excel(file_paths[0], sheet_name=None)

sheet_names = list(all_sheets.keys())
print(f"Sheet Names: {sheet_names}")

Sheet Names: ['Daily Company Info Summary', 'Company Listing', 'Ticker Volatility', 'Stock History', 'Stock Intraday', 'Stock Rating']


In [229]:
dfs = {file_path: pd.read_excel(file_path, sheet_name='Stock History') for file_path in file_paths}

df = pd.concat(dfs.values(), keys=dfs.keys())

# Reset the index
df.rename(columns={'Time': 'Date'}, inplace=True)
df.sort_values(["Symbol", "Date"], inplace=True)
df.reset_index(drop=True, inplace=True)

df

,Symbol,Date,Open,High,Low,Close,Volume
0,A32,2023-12-28,30000,30000,30000,30000,100
1,AAA,2023-12-25,9250,9350,9250,9330,1554200
2,AAA,2023-12-26,9350,9390,9300,9310,1322600
3,AAA,2023-12-27,9340,9490,9340,9400,1999000
4,AAA,2023-12-28,9420,9540,9400,9420,1452800
...,...,...,...,...,...,...,...
335,BHI,2023-12-27,16800,17500,16800,17500,900
336,BHI,2023-12-28,19400,19400,19400,19400,800
337,BHK,2023-12-27,12900,12900,12900,12900,200
338,BHK,2023-12-28,14100,14100,14100,14100,1100


In [241]:
def ft_min(x, n):
    closing_prices = x['Close'].tail(n).values
    
    if len(closing_prices) == 0:
        return x 
    
    fft_result = np.fft.fft(closing_prices)
    
    m = len(closing_prices)
    frequencies = np.fft.fftfreq(m, d=1)
    
    non_zero_frequencies = frequencies[np.nonzero(frequencies)]
    
    if len(non_zero_frequencies) > 0:
        max_frequency = max(np.abs(non_zero_frequencies))
        x['FT_Max'] = min_frequency
    else:
        x['FT_Max'] = np.nan
    
    return x

n = 6
df = df.groupby("Symbol").apply(lambda x: ft_min(x, n)).reset_index(0, drop=True).fillna(0)
df['FT_Max'].value_counts()

FT_Max
0.2    333
0.0      7
Name: count, dtype: int64

In [246]:
def ft_mean(x, n):
    closing_prices = x['Close'].tail(n).values
    
    if len(closing_prices) == 0:
        return x 
    
    fft_result = np.fft.fft(closing_prices)
    
    m = len(closing_prices)
    frequencies = np.fft.fftfreq(m, d=1)
    
    non_zero_frequencies = frequencies[np.nonzero(frequencies)]
    
    if len(non_zero_frequencies) > 0:
        mean_frequency = np.mean(np.abs(non_zero_frequencies))
        x['FT_Mean'] = mean_frequency
    else:
        x['FT_Mean'] = np.nan
    
    return x

n = 5
df = df.groupby("Symbol").apply(lambda x: ft_mean(x, n)).reset_index(0, drop=True).fillna(0)
df

,Symbol,Date,Open,High,Low,Close,Volume,FT_Min,FT_Max,FT_Mean
0,A32,2023-12-28,30000,30000,30000,30000,100,0.000000,0.0,0.000000
1,AAA,2023-12-25,9250,9350,9250,9330,1554200,0.200000,0.2,0.300000
2,AAA,2023-12-26,9350,9390,9300,9310,1322600,0.200000,0.2,0.300000
3,AAA,2023-12-27,9340,9490,9340,9400,1999000,0.200000,0.2,0.300000
4,AAA,2023-12-28,9420,9540,9400,9420,1452800,0.200000,0.2,0.300000
...,...,...,...,...,...,...,...,...,...,...
335,BHI,2023-12-27,16800,17500,16800,17500,900,0.250000,0.2,0.333333
336,BHI,2023-12-28,19400,19400,19400,19400,800,0.250000,0.2,0.333333
337,BHK,2023-12-27,12900,12900,12900,12900,200,0.333333,0.2,0.333333
338,BHK,2023-12-28,14100,14100,14100,14100,1100,0.333333,0.2,0.333333


In [179]:
def obv(x):
    obv = ((x["Close"] > x["Close"].shift()).astype(int) - (x["Close"] < x["Close"].shift()).astype(int)) * x["Volume"] + x['OBV'].shift()
    return obv

df['OBV'] = 0
df['OBV'] = df.groupby("Symbol").apply(lambda x: obv(x)).reset_index(level=0, drop=True).fillna(0)
df

,Symbol,Date,Open,High,Low,Close,Volume,OBV
0,A32,2023-12-28,30000,30000,30000,30000,100,0.0
1,AAA,2023-12-25,9250,9350,9250,9330,1554200,0.0
2,AAA,2023-12-26,9350,9390,9300,9310,1322600,-1322600.0
3,AAA,2023-12-27,9340,9490,9340,9400,1999000,1999000.0
4,AAA,2023-12-28,9420,9540,9400,9420,1452800,1452800.0
...,...,...,...,...,...,...,...,...
335,BHI,2023-12-27,16800,17500,16800,17500,900,0.0
336,BHI,2023-12-28,19400,19400,19400,19400,800,800.0
337,BHK,2023-12-27,12900,12900,12900,12900,200,0.0
338,BHK,2023-12-28,14100,14100,14100,14100,1100,1100.0


In [151]:
df['OBV'].value_counts()

Series([], Name: count, dtype: int64)

In [144]:
import numpy as np
def calculate_min_frequency(df, n):
    closing_prices = df['Close'].tail(n).values

    # Check for NaN values and replace with the mean
    closing_prices = np.nan_to_num(closing_prices, nan=np.nanmean(closing_prices))

    fourier_transform = np.fft.fft(closing_prices)
    frequencies = np.fft.fftfreq(len(fourier_transform))
    magnitude_spectrum = np.abs(fourier_transform)

    # Exclude the zero frequency component and very low frequencies
    valid_indices = np.where((frequencies > 0.01) & (magnitude_spectrum > 0.01))
    nonzero_frequencies = frequencies[valid_indices]
    nonzero_magnitudes = magnitude_spectrum[valid_indices]

    if len(nonzero_frequencies) == 0:
        return np.nan

    min_frequency = nonzero_frequencies[np.argmin(nonzero_magnitudes)]
    return min_frequency


# Specify the number of past days to consider
n_days = 30

# Calculate the minimum frequency for each symbol (assuming there is a 'Symbol' column)
df['FTmin'] = df.groupby('Symbol').apply(lambda x: calculate_min_frequency(x, n_days))
df['FTmin'].value_counts()

Series([], Name: count, dtype: int64)

In [128]:
df['PercentageChange'] = df.groupby('Symbol')['Close'].pct_change(2).fillna(0)
df

,Symbol,Date,Open,High,Low,Close,Volume,PercentageChange
0,A32,2023-12-28,30000,30000,30000,30000,100,0.000000
1,AAA,2023-12-25,9250,9350,9250,9330,1554200,0.000000
2,AAA,2023-12-26,9350,9390,9300,9310,1322600,0.000000
3,AAA,2023-12-27,9340,9490,9340,9400,1999000,0.007503
4,AAA,2023-12-28,9420,9540,9400,9420,1452800,0.011815
...,...,...,...,...,...,...,...,...
335,BHI,2023-12-27,16800,17500,16800,17500,900,0.000000
336,BHI,2023-12-28,19400,19400,19400,19400,800,0.108571
337,BHK,2023-12-27,12900,12900,12900,12900,200,0.000000
338,BHK,2023-12-28,14100,14100,14100,14100,1100,0.000000


In [133]:
df['LowestClose'] = df.groupby('Symbol')['Close'].transform(lambda x: x.rolling(window=3).min())
df['PercentageChangeToLowest'] = (df['Close'] - df['LowestClose']) / df['Close']
df

,Symbol,Date,Open,High,Low,Close,Volume,LowestClose,PercentageChangeToLowest
0,A32,2023-12-28,30000,30000,30000,30000,100,NaN,NaN
1,AAA,2023-12-25,9250,9350,9250,9330,1554200,NaN,NaN
2,AAA,2023-12-26,9350,9390,9300,9310,1322600,NaN,NaN
3,AAA,2023-12-27,9340,9490,9340,9400,1999000,9310.0,0.009574
4,AAA,2023-12-28,9420,9540,9400,9420,1452800,9310.0,0.011677
...,...,...,...,...,...,...,...,...,...
335,BHI,2023-12-27,16800,17500,16800,17500,900,17500.0,0.000000
336,BHI,2023-12-28,19400,19400,19400,19400,800,17500.0,0.097938
337,BHK,2023-12-27,12900,12900,12900,12900,200,NaN,NaN
338,BHK,2023-12-28,14100,14100,14100,14100,1100,NaN,NaN


In [267]:
df["Median"] = df.groupby('Symbol')['Volume'].rolling(3, min_periods=1).median().reset_index(0, drop=True)
df[df["Symbol"] == "AAA"]

,Symbol,Date,Open,High,Low,Close,Volume,Price_Change,Median
1,AAA,2023-12-25,9250,9350,9250,9330,1554200,0.0,1554200.0
2,AAA,2023-12-26,9350,9390,9300,9310,1322600,-20.0,1438400.0
3,AAA,2023-12-27,9340,9490,9340,9400,1999000,90.0,1554200.0
4,AAA,2023-12-28,9420,9540,9400,9420,1452800,20.0,1452800.0
5,AAA,2023-12-29,9440,9550,9430,9450,1660900,30.0,1660900.0


In [264]:
df.sort_values(["Symbol", "Date"], inplace=True)
df["Price_Change"] = df.groupby("Symbol")["Close"].diff().fillna(0)
df[df["Symbol"] == "AAA"]

,Symbol,Date,Open,High,Low,Close,Volume,Price_Change
1,AAA,2023-12-25,9250,9350,9250,9330,1554200,0.0
2,AAA,2023-12-26,9350,9390,9300,9310,1322600,-20.0
3,AAA,2023-12-27,9340,9490,9340,9400,1999000,90.0
4,AAA,2023-12-28,9420,9540,9400,9420,1452800,20.0
5,AAA,2023-12-29,9440,9550,9430,9450,1660900,30.0


In [183]:
df.sort_values(["Symbol", "Date"], inplace=True)
df["SMA_2_Days"] = df.groupby("Symbol")["Close"].rolling(window=2).mean().reset_index(0, drop=True).fillna(0)
df[df["Symbol"] == "AAA"]

,Symbol,Date,Open,High,Low,Close,Volume,Price_Change,SMA_2_Days
1,AAA,2023-12-25,9250,9350,9250,9330,1554200,0.0,0.0
2,AAA,2023-12-26,9350,9390,9300,9310,1322600,-20.0,9320.0
3,AAA,2023-12-27,9340,9490,9340,9400,1999000,90.0,9355.0
4,AAA,2023-12-28,9420,9540,9400,9420,1452800,20.0,9410.0
5,AAA,2023-12-29,9440,9550,9430,9450,1660900,30.0,9435.0


In [184]:
df.sort_values(["Symbol", "Date"], inplace=True)
df["WMA_2_Days"] =  df.groupby("Symbol")["Close"].rolling(window=5).apply(lambda x: (x * range(1, len(x) + 1)).sum() / 2, raw=True).reset_index(0, drop=True).fillna(0)
df[df["Symbol"] == "AAA"]

,Symbol,Date,Open,High,Low,Close,Volume,Price_Change,SMA_2_Days,WMA_2_Days
1,AAA,2023-12-25,9250,9350,9250,9330,1554200,0.0,0.0,0.0
2,AAA,2023-12-26,9350,9390,9300,9310,1322600,-20.0,9320.0,0.0
3,AAA,2023-12-27,9340,9490,9340,9400,1999000,90.0,9355.0,0.0
4,AAA,2023-12-28,9420,9540,9400,9420,1452800,20.0,9410.0,0.0
5,AAA,2023-12-29,9440,9550,9430,9450,1660900,30.0,9435.0,70540.0


In [194]:
df.sort_values(["Symbol", "Date"], inplace=True)
df["EMA_2_Days"] = df.groupby("Symbol")["Close"].transform(lambda x: x.ewm(span=2).mean())
df.head(10)

,Symbol,Date,Open,High,Low,Close,Volume,Price_Change,SMA_2_Days,WMA_2_Days,EMA_2_Days
0,A32,2023-12-28,30000,30000,30000,30000,100,0.0,0.0,0.0,30000.000000
1,AAA,2023-12-25,9250,9350,9250,9330,1554200,0.0,0.0,0.0,9330.000000
2,AAA,2023-12-26,9350,9390,9300,9310,1322600,-20.0,9320.0,0.0,9315.000000
3,AAA,2023-12-27,9340,9490,9340,9400,1999000,90.0,9355.0,0.0,9373.846154
4,AAA,2023-12-28,9420,9540,9400,9420,1452800,20.0,9410.0,0.0,9405.000000
5,AAA,2023-12-29,9440,9550,9430,9450,1660900,30.0,9435.0,70540.0,9435.123967
6,AAM,2023-12-25,9210,9390,9210,9390,2000,0.0,0.0,0.0,9390.000000
7,AAM,2023-12-26,9390,9390,9300,9370,2700,-20.0,9380.0,0.0,9375.000000
8,AAM,2023-12-27,9370,9370,9100,9100,2200,-270.0,9235.0,0.0,9184.615385
9,AAM,2023-12-28,9120,9360,9100,9360,3400,260.0,9230.0,0.0,9303.000000


In [219]:
timeframe = 3
df['HH'] = df.groupby('Symbol')['Close'].rolling(window=timeframe).max().reset_index(0, drop=True)
df['LL'] = df.groupby('Symbol')['Close'].rolling(window=timeframe).min().reset_index(0, drop=True)

df['%K'] = 100 * (df['Close'] - df['LL']) / (df['HH'] - df['LL'])

df['%D'] = df['%K'].rolling(window=3).mean() 
df.fillna(0, inplace=True)
df[df["Symbol"] == "ALT"]

,Symbol,Date,Open,High,Low,Close,Volume,HH,LL,%K,%D
143,ALT,2023-12-25,13500,13500,13100,13200,8200,0.0,0.0,0.0,0.000000
144,ALT,2023-12-26,13400,13500,13400,13400,15300,0.0,0.0,0.0,0.000000
145,ALT,2023-12-27,13400,13400,13400,13400,6800,13400.0,13200.0,100.0,0.000000
146,ALT,2023-12-28,13200,13200,13200,13200,200,13400.0,13200.0,0.0,0.000000
147,ALT,2023-12-29,13400,13400,13400,13400,300,13400.0,13200.0,100.0,66.666667


In [278]:
df

,Symbol,Date,Open,High,Low,Close,Volume,change,gain,loss
0,A32,2023-12-28,30000,30000,30000,30000,100,NaN,NaN,NaN
1,AAA,2023-12-25,9250,9350,9250,9330,1554200,-20670.0,0.0,20670.0
2,AAA,2023-12-26,9350,9390,9300,9310,1322600,-20.0,0.0,20.0
3,AAA,2023-12-27,9340,9490,9340,9400,1999000,90.0,90.0,0.0
4,AAA,2023-12-28,9420,9540,9400,9420,1452800,20.0,20.0,0.0
...,...,...,...,...,...,...,...,...,...,...
335,BHI,2023-12-27,16800,17500,16800,17500,900,0.0,0.0,-0.0
336,BHI,2023-12-28,19400,19400,19400,19400,800,1900.0,1900.0,0.0
337,BHK,2023-12-27,12900,12900,12900,12900,200,-6500.0,0.0,6500.0
338,BHK,2023-12-28,14100,14100,14100,14100,1100,1200.0,1200.0,0.0


In [284]:
n=3

df['change'] = df['Close'].diff()
df['gain'] = df.change.mask(df.change < 0, 0.0)
df['loss'] = -df.change.mask(df.change > 0, -0.0)

def rma(x, n):
    a = np.full_like(x, np.nan)
    if len(a) < n:
        return a
    a[n-1] = x[:n].mean()
    for i in range(n, len(x)):
        a[i] = (a[i-1] * (n - 1) + x[i]) / n
    return a

df['avg_gain'] = df.groupby("Symbol")["gain"].transform(lambda x: rma(x.to_numpy(), n))
df['avg_loss'] = df.groupby("Symbol")["loss"].transform(lambda x: rma(x.to_numpy(), n))

df['rs'] = df['avg_gain']/df['avg_loss']
df['rsi'] = 100 - (100 / (1 + df['rs']))
df[df["Symbol"] == "AAA"]

,Symbol,Date,Open,High,Low,Close,Volume,change,gain,loss,avg_gain,avg_loss,rs,rsi
1,AAA,2023-12-25,9250,9350,9250,9330,1554200,-20670.0,0.0,20670.0,NaN,NaN,NaN,NaN
2,AAA,2023-12-26,9350,9390,9300,9310,1322600,-20.0,0.0,20.0,NaN,NaN,NaN,NaN
3,AAA,2023-12-27,9340,9490,9340,9400,1999000,90.0,90.0,0.0,30.000000,6896.666667,0.004350,0.433109
4,AAA,2023-12-28,9420,9540,9400,9420,1452800,20.0,20.0,0.0,26.666667,4597.777778,0.005800,0.576646
5,AAA,2023-12-29,9440,9550,9430,9450,1660900,30.0,30.0,0.0,27.777778,3065.185185,0.009062,0.898096


In [285]:
def williams_percent_r(df, n=14):
    df['Highest High'] = df['High'].rolling(window=n, min_periods=1).max()
    df['Lowest Low'] = df['Low'].rolling(window=n, min_periods=1).min()
    df['Williams %R'] = (df['Highest High'] - df['Close']) / (df['Highest High'] - df['Lowest Low']) * -100
    df.drop(['Highest High', 'Lowest Low'], axis=1, inplace=True)
 

williams_percent_r(df)
df

,Symbol,Date,Open,High,Low,Close,Volume,change,gain,loss,avg_gain,avg_loss,rs,rsi,Highest High,Lowest Low,Williams %R
0,A32,2023-12-28,30000,30000,30000,30000,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30000.0,30000.0,NaN
1,AAA,2023-12-25,9250,9350,9250,9330,1554200,-20670.0,0.0,20670.0,NaN,NaN,NaN,NaN,30000.0,9250.0,-99.614458
2,AAA,2023-12-26,9350,9390,9300,9310,1322600,-20.0,0.0,20.0,NaN,NaN,NaN,NaN,30000.0,9250.0,-99.710843
3,AAA,2023-12-27,9340,9490,9340,9400,1999000,90.0,90.0,0.0,30.000000,6896.666667,0.004350,0.433109,30000.0,9250.0,-99.277108
4,AAA,2023-12-28,9420,9540,9400,9420,1452800,20.0,20.0,0.0,26.666667,4597.777778,0.005800,0.576646,30000.0,9250.0,-99.180723
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335,BHI,2023-12-27,16800,17500,16800,17500,900,0.0,0.0,-0.0,933.333333,0.000000,inf,100.000000,25500.0,12800.0,-62.992126
336,BHI,2023-12-28,19400,19400,19400,19400,800,1900.0,1900.0,0.0,1255.555556,0.000000,inf,100.000000,25500.0,13000.0,-48.800000
337,BHK,2023-12-27,12900,12900,12900,12900,200,-6500.0,0.0,6500.0,NaN,NaN,NaN,NaN,25500.0,12900.0,-100.000000
338,BHK,2023-12-28,14100,14100,14100,14100,1100,1200.0,1200.0,0.0,NaN,NaN,NaN,NaN,25500.0,12900.0,-90.476190


In [31]:
def calculate_money_flow_and_ad(group):
    group['MoneyFlow'] = ((group['Close'] - group['Low']) - (group['High'] - group['Close'])) / (group['High'] - group['Low'])
    group['AD'] = group['MoneyFlow'].shift(1) + group['MoneyFlow']
    return group['AD']

df['AD'] = df.groupby('Symbol').apply(calculate_money_flow_and_ad).reset_index(0, drop=True).fillna(0)

In [32]:
df

,Symbol,Date,Open,High,Low,Close,Volume,AD,MoneyFlow
0,A32,2023-12-28,30000,30000,30000,30000,100,0.000000,0.000000
1,AAA,2023-12-25,9250,9350,9250,9330,1554200,0.000000,0.600000
2,AAA,2023-12-26,9350,9390,9300,9310,1322600,-0.177778,-0.777778
3,AAA,2023-12-27,9340,9490,9340,9400,1999000,-0.977778,-0.200000
4,AAA,2023-12-28,9420,9540,9400,9420,1452800,-0.914286,-0.714286
...,...,...,...,...,...,...,...,...,...
335,BHI,2023-12-27,16800,17500,16800,17500,900,0.000000,1.000000
336,BHI,2023-12-28,19400,19400,19400,19400,800,0.000000,0.000000
337,BHK,2023-12-27,12900,12900,12900,12900,200,0.000000,0.000000
338,BHK,2023-12-28,14100,14100,14100,14100,1100,0.000000,0.000000


In [83]:
def calculate_cci(group, window=2):
    typical_price = (group['High'] + group['Low'] + group['Close']) / 3
    sma_typical_price = typical_price.rolling(window=window).mean()
    mean_deviation = typical_price.rolling(window=window).apply(lambda x: abs(x - x.mean()).mean(), raw=True)
    cci = (typical_price - sma_typical_price) / (0.015 * mean_deviation)
    group['CCI'] = cci
    return group['CCI']

df['CCI'] = df.groupby('Symbol').apply(calculate_cci).reset_index(0, drop=True).fillna(0)
df

,Symbol,Date,Open,High,Low,Close,Volume,CCI
0,A32,2023-12-28,30000,30000,30000,30000,100,0.000000
1,AAA,2023-12-25,9250,9350,9250,9330,1554200,0.000000
2,AAA,2023-12-26,9350,9390,9300,9310,1322600,66.666667
3,AAA,2023-12-27,9340,9490,9340,9400,1999000,66.666667
4,AAA,2023-12-28,9420,9540,9400,9420,1452800,66.666667
...,...,...,...,...,...,...,...,...
335,BHI,2023-12-27,16800,17500,16800,17500,900,-66.666667
336,BHI,2023-12-28,19400,19400,19400,19400,800,66.666667
337,BHK,2023-12-27,12900,12900,12900,12900,200,0.000000
338,BHK,2023-12-28,14100,14100,14100,14100,1100,66.666667


In [120]:
def calculate_cci(group, window=5):
    group['TypicalPrice'] = (group['High'] + group['Low'] + group['Close']) / 3
    group['SMA'] = group['TypicalPrice'].rolling(window=window).mean()

    group['MeanDeviation'] = group['TypicalPrice'].rolling(window=window).apply(lambda x: abs(x.mean() - x).mean(), raw=True)

    group['CCI'] = (group['TypicalPrice'] - group['SMA']) / (0.015 * group['MeanDeviation'])
    return group['CCI']

df['CCI'] = df.groupby('Symbol').apply(calculate_cci).T.reset_index(0, drop=True).fillna(0)
df

,Symbol,Date,Open,High,Low,Close,Volume,CCI
0,A32,2023-12-28,30000,30000,30000,30000,100,0.0
1,AAA,2023-12-25,9250,9350,9250,9330,1554200,0.0
2,AAA,2023-12-26,9350,9390,9300,9310,1322600,0.0
3,AAA,2023-12-27,9340,9490,9340,9400,1999000,0.0
4,AAA,2023-12-28,9420,9540,9400,9420,1452800,0.0
...,...,...,...,...,...,...,...,...
335,BHI,2023-12-27,16800,17500,16800,17500,900,0.0
336,BHI,2023-12-28,19400,19400,19400,19400,800,0.0
337,BHK,2023-12-27,12900,12900,12900,12900,200,0.0
338,BHK,2023-12-28,14100,14100,14100,14100,1100,0.0


In [121]:
df["CCI"].value_counts()

CCI
 0.000000      293
 88.888889       1
 166.666667      1
 57.493540       1
 4.385965        1
 127.107652      1
-30.303030       1
 48.941799       1
 96.205962       1
-41.666667       1
 93.495935       1
-83.333333       1
 74.433657       1
 131.043257      1
 29.914530       1
-40.123457       1
 44.025157       1
 55.555556       1
 18.518519       1
-66.893424       1
 78.431373       1
 161.172161      1
-102.272727      1
-55.555556       1
-45.977011       1
 46.875000       1
 68.181818       1
-71.651090       1
-20.833333       1
 156.565657      1
-55.555556       1
 83.333333       1
-141.025641      1
 95.238095       1
 61.797753       1
-25.862069       1
 121.212121      1
-60.606061       1
 37.037037       1
-10.204082       1
-94.017094       1
 76.208710       1
-14.814815       1
-47.101449       1
-55.555556       1
 166.666667      1
 28.169014       1
 13.757524       1
Name: count, dtype: int64